# Part One

In [14]:
def getPuzzleInput():
	with open("Day 9.txt") as file:
		return file.read()

def getArea(point1, point2):
	return (abs(point1[0] - point2[0]) + 1) * (abs(point1[1] - point2[1]) + 1)
	

In [15]:
points = [(int(x.split(",")[0]), int(x.split(",")[1])) for x in getPuzzleInput().split("\n")]
pointsCombinations = []

for i in range(len(points)):
	point1 = points[i]
	for j in range(len(points)):
		if i == j:
			continue
		point2 = points[j]
		pointsCombinations.append((point1, point2, getArea(point1, point2)))
pointsCombinations.sort(key=lambda x: x[2], reverse=True)

print(pointsCombinations[0][2])

4767418746


4767418746

In [3]:
# one liner for fun
print(max([getArea(tuple(map(int, points[i].split(","))), tuple(map(int, points[j].split(",")))) for i in range(len(getPuzzleInput().split('\n'))) for j in range(len(getPuzzleInput().split('\n'))) if i != j]))

4767418746


# Part Two

In [ ]:
import numpy as np

points = [(int(x.split(",")[0]), int(x.split(",")[1])) for x in getPuzzleInput().split("\n")]

maxRow = max([line[0] for line in points])
maxCol = max([line[1] for line in points])

outerBorderPoints = []

grid = np.full((maxRow+2, maxCol+2), ".")

for line in points:
	grid[line[0]][line[1]] = "#"

# print(grid)

def printGrid(grid):
	# counterclockwise and flipped along the horizontal axis
	# printingGrid = np.flipud(np.rot90(grid))
	printingGrid = grid
	for row in range(len(printingGrid)):
		for col in range(len(printingGrid[row])):
			print(printingGrid[row][col], end=" ")
		print()

# printGrid(grid)

def createBorders(grid):
	for i in range(len(points)): # so it wraps
		x1, y1 = list(points[i%len(points)])
		x2, y2 = list(points[(i+1)%len(points)])

		# going up
		if x1 < x2:
			# account for exclusive slices and 0 width slices
			x1 += 1
			y2 += 1
			grid[x1:x2, y1:y2] = "X"
			outerBorderPoints.extend([(x,y) for x in range(x1-2, x2+2) for y in range(y1-1, y2-1)])
			# print(x1,y1,x2,y2)
			# print("up:", [(x,y) for x in range(x1-2, x2+2) for y in range(y1-1, y2-1)])
			# print()
		
		# going right
		elif y1 < y2:
			x2 += 1
			y1 += 1
			grid[x1:x2, y1:y2] = "X"
			outerBorderPoints.extend([(x,y) for x in range(x1+1, x2+1) for y in range(y1-2, y2+2)])
			# print(x1,y1,x2,y2)
			# print("right:", [(x,y) for x in range(x1+1, x2+1) for y in range(y1-2, y2+2)])
			# print()

		# going down
		elif x1 > x2:
			x2 += 1
			y2 += 1
			# swap xs so we're not going backwards
			grid[x2:x1, y1:y2] = "X"
			outerBorderPoints.extend([(x,y) for x in range(x2-2, x1+2) for y in range(y1+1, y2+1)])
			# print(x1,y1,x2,y2)
			# print("down:", [(x,y) for x in range(x2-2, x1+2) for y in range(y1+1, y2+1)])
			# print()

		# going left
		elif y1 > y2:
			x2 += 1
			y2 += 1
			# swap ys so we're not going backward
			grid[x1:x2, y2:y1] = "X"
			outerBorderPoints.extend([(x,y) for x in range(x1-1, x2-1) for y in range(y2-2, y1+2)])
			# print(x1,y1,x2,y2)
			# print("left:", [(x,y) for x in range(x1-1, x2-1) for y in range(y2-2, y1+2)])
			# print()
	
	return grid

grid = createBorders(grid)

In [4]:
def dedupe(arr: list[tuple]) -> list:
	newArr = []
	for a in arr:
		if a not in newArr:
			newArr.append(a)
	return newArr

In [5]:
import matplotlib.path as mpath

# The outer corners of the shape if you expanded the perimeter by one
corners = dedupe([c for c in outerBorderPoints if outerBorderPoints.count(c) > 1])
corners.append(corners[0]) # to complete the path for mpath to use

# create a path between the corners
outerPath = mpath.Path(corners)

largestArea = 0

# Go through all of the possible combinations of points around the shape
for i in range(len(pointsCombinations)):
	pc1, pc2 = pointsCombinations[i][0:2]
	
	# Create a path from the rectangles opposite points
	rectPoints = [pc1, (pc1[0], pc2[1]), pc2, (pc2[0], pc1[1]), pc1]
	rectPath = mpath.Path(rectPoints)

	# If the path of the rectangle is completely contained within the path the corners make
	if outerPath.contains_path(rectPath):
		# And it's area is the largest we've seen
		if (a := getArea(pc1, pc2)) > largestArea:
			# Ladies and gentlemen, we got 'em.
			largestArea = a

print(largestArea)

4635268638


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.path as mpath

# outerBorderPoints = np.array(outerBorderPoints)
# print(outerBorderPoints)
print("orig corners:", outerBorderPoints)
corners = dedupe([c for c in outerBorderPoints if outerBorderPoints.count(c) > 1])
# for c in outerBorderPoints:
# 	# print("count:", c, outerBorderPoints.count(c))
# 	if outerBorderPoints.count(c) > 1:
# 		# print("count:", c, outerBorderPoints.count(c))

corners.append(corners[0]) # to complete the path for mpath to use
print("corners:", corners)
print("points:", points)

# path = mpath.Path(corners)
outerPath = mpath.Path(corners)
innerPath = mpath.Path(points)

pointsContained = outerPath.contains_path(innerPath)
print("contained?", pointsContained)

pc1, pc2 = pointsCombinations[0][0:2]
print(pc1, pc2)

rectPoints = [pc1, (pc1[0], pc2[1]), pc2, (pc2[0], pc1[1]), pc1]
print(rectPoints)
rectPath = mpath.Path(rectPoints)
pointsContained = outerPath.contains_path(rectPath)
print("contained 2?", pointsContained)

r1 = list(range(pc2[0], pc1[0]+1))
r2 = list(range(pc1[1], pc2[1]+1))
print(r1, r2)

x, y = np.meshgrid(r1, r2)
# print(x,y)
rect = np.vstack((x.flatten(), y.flatten())).T
print("rect points:", rect)

contained = outerPath.contains_points(rect)
print("contained:", contained)

contained2dMask = contained.reshape(len(r1), len(r2))

col = np.where(contained, 'g', 'r')
# print(col)

# xData = rect[:, 0]
# yData = rect[:, 1]

# plt.figure(figsize=(5,5))
# plt.xlim(0, 14)
# plt.ylim(0, 14)
# plt.grid(True)
# plt.scatter(xData, yData, c=col, marker=".", linewidths=2)
# plt.show()

# # xData = [x[0] for x in points]
# # yData = [x[1] for x in points]
# xData = [x[0] for x in corners]
# yData = [x[1] for x in corners]

# plt.figure(figsize=(5,5))
# plt.xlim(0, 14)
# plt.ylim(0, 14)
# plt.grid(True)
# plt.scatter(xData, yData, marker=".", linewidths=2)
# plt.show()

orig corners: [(6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0), (12, 0), (12, 1), (12, 2), (12, 3), (12, 4), (12, 5), (12, 6), (12, 7), (12, 8), (8, 8), (9, 8), (10, 8), (11, 8), (12, 8), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (1, 6), (2, 6), (3, 6), (4, 6), (5, 6), (6, 6), (7, 6), (8, 6), (9, 6), (10, 6), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 2), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4)]
corners: [(6, 0), (12, 0), (12, 8), (8, 8), (8, 6), (1, 6), (1, 2), (6, 2), (6, 0)]
points: [(7, 1), (11, 1), (11, 7), (9, 7), (9, 5), (2, 5), (2, 3), (7, 3)]
contained? True
(11, 1) (2, 5)
[(11, 1), (11, 5), (2, 5), (2, 1), (11, 1)]
contained 2? False
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11] [1, 2, 3, 4, 5]
rect points: [[ 2  1]
 [ 3  1]
 [ 4  1]
 [ 5  1]
 [ 6  1]
 [ 7  1]
 [ 8  1]
 [ 9  1]
 [10  1]
 [11  1]
 [ 2  2]
 [ 3  2]
 [ 4  2]
 [ 5  2]
 [ 6  2]
 [ 7  2]
 [ 8  2]
 [ 9  2]
 [10  2]
 [11  2]
 [ 2  3]
 [ 3  3]
 [ 4  3]
 [ 5  3]
 [ 6  

too high:
 - 4635268638